## Gcn4 peak analysis

### Running Bash commands inside a Jupyter notebook
To run Bash commands in a Jupyter notebook, you need to install the Bash kernel for Jupyter. For that, run the following commands in the terminal:
```
pip install bash_kernel
python -m bash_kernel.install
```

### Check the raw data

We have a list of Gcn4 peaks (from in vivo/in vitro experiments) that we want to compare. Some peaks are stored in BED files, some peaks are stored in narrowPeak files (output from MACS2).

In [1]:
# List the available BED files and narrowPeak files
ls -lh raw_data/

total 25448
-rwxrwxrwx  1 cherejirv  NIH\Domain Users    29K Mar 25 15:51 Gcn4_ChIP-seq_peaks.bed
-rwxrwxrwx  1 cherejirv  NIH\Domain Users    97K Mar 25 15:51 Gcn4_motifs.bed
-rwxrwxrwx  1 cherejirv  NIH\Domain Users    11K Mar 25 15:51 Round1_peaks.narrowPeak
-rwxrwxrwx  1 cherejirv  NIH\Domain Users   158K Mar 25 15:51 Round2_peaks.narrowPeak
-rwxrwxrwx  1 cherejirv  NIH\Domain Users   228K Mar 25 15:51 Round3_peaks.narrowPeak
-rwxr-x---  1 cherejirv  NIH\Domain Users    12M Mar 26 15:45 sacCer3.fa


In [2]:
# Check the head/tail of one of some these files
head -n 5 raw_data/Gcn4_ChIP-seq_peaks.bed

chrI	18235	18470	Name=Peak_1;Nearest_TSS=YAL064W
chrI	19302	19696	Name=Peak_2;Nearest_TSS=YAL064W
chrI	22053	22364	Name=Peak_3;Nearest_TSS=YAL063C-A
chrI	54820	55264	Name=Peak_4;Nearest_TSS=YAL048C
chrI	75011	75257	Name=Peak_5;Nearest_TSS=YAL036C


In [3]:
tail -n 5 raw_data/Round1_peaks.narrowPeak

chrXVI	62090	62195	Round1_peak_150	389	.	4.14545	42.20868	38.91930	48
chrXVI	497444	497563	Round1_peak_151	597	.	5.19936	64.26705	59.77417	59
chrXVI	625273	625399	Round1_peak_152	682	.	6.42615	73.55953	68.24533	67
chrXVI	626000	626100	Round1_peak_153	432	.	4.68250	46.74600	43.24176	46
chrXVI	747290	747400	Round1_peak_154	519	.	4.90535	55.89925	51.93674	57


You can read more about the narrowPeak format here:  
https://genome.ucsc.edu/FAQ/FAQformat.html#format12

You can read more about the possible columns in a BED file here:  
https://genome.ucsc.edu/FAQ/FAQformat.html#format1

The raw files contain different numbers of rows. Let's make these look nicer by keeping only the rows that matter (chromosome name, start position, end position).

In [4]:
# Make a new folder
mkdir -p data

# Remove the extra columns, and keep only the required first 3 columns
cut -f 1,2,3 raw_data/Gcn4_ChIP-seq_peaks.bed | sort -k1,1V -k2,2n -k3,3n - > data/Gcn4_ChIP_Rawal_et_al.bed
cut -f 1,2,3 raw_data/Gcn4_motifs.bed | sort -k1,1V -k2,2n -k3,3n - > data/Gcn4_motifs_Rawal_et_al.bed
cut -f 1,2,3 raw_data/Round1_peaks.narrowPeak | sort -k1,1V -k2,2n -k3,3n - > data/Gcn4_SELEX_1.bed
cut -f 1,2,3 raw_data/Round2_peaks.narrowPeak | sort -k1,1V -k2,2n -k3,3n - > data/Gcn4_SELEX_2.bed
cut -f 1,2,3 raw_data/Round3_peaks.narrowPeak | sort -k1,1V -k2,2n -k3,3n - > data/Gcn4_SELEX_3.bed

In [5]:
# Check the new files
head -n 5 data/*

==> data/Gcn4_ChIP_Rawal_et_al.bed <==
chrI	18235	18470
chrI	19302	19696
chrI	22053	22364
chrI	54820	55264
chrI	75011	75257

==> data/Gcn4_SELEX_1.bed <==
chr2um	76	634
chr2um	819	1523
chr2um	1643	2085
chr2um	2170	2751
chr2um	2855	2968

==> data/Gcn4_SELEX_2.bed <==
chr2um	79	621
chr2um	858	1054
chr2um	1198	1462
chr2um	1642	2096
chr2um	2253	2733

==> data/Gcn4_SELEX_3.bed <==
chr2um	140	625
chr2um	1188	1325
chr2um	1647	2059
chr2um	2270	2732
chr2um	3479	3774

==> data/Gcn4_motifs_Rawal_et_al.bed <==
chrI	13673	13683
chrI	18324	18334
chrI	19487	19497
chrI	22216	22226
chrI	33630	33640


We see that some SELEX peaks were mapped to the 2 micron plasmid. Some peaks are found on chrMT. Let's remove all these.

In [6]:
grep -v '^chr2um' data/Gcn4_SELEX_1.bed |
    grep -v '^chrMT' - > data/Gcn4_SELEX_1_wo_2micron_and_mito.bed
    
grep -v '^chr2um' data/Gcn4_SELEX_2.bed |
    grep -v '^chrMT' - > data/Gcn4_SELEX_2_wo_2micron_and_mito.bed
    
grep -v '^chr2um' data/Gcn4_SELEX_3.bed |
    grep -v '^chrMT' - > data/Gcn4_SELEX_3_wo_2micron_and_mito.bed

In [7]:
# Rename these file to simplify our life (overwrite the old files)
mv data/Gcn4_SELEX_1_wo_2micron_and_mito.bed data/Gcn4_SELEX_1.bed
mv data/Gcn4_SELEX_2_wo_2micron_and_mito.bed data/Gcn4_SELEX_2.bed
mv data/Gcn4_SELEX_3_wo_2micron_and_mito.bed data/Gcn4_SELEX_3.bed

In [8]:
# Check the new files
head -n 5 data/Gcn4_SELEX_?.bed

==> data/Gcn4_SELEX_1.bed <==
chrI	22164	22293
chrI	80242	80355
chrI	115563	115684
chrII	33917	34027
chrII	254305	254427

==> data/Gcn4_SELEX_2.bed <==
chrI	12978	13078
chrI	18257	18382
chrI	19406	19567
chrI	22148	22306
chrI	25112	25248

==> data/Gcn4_SELEX_3.bed <==
chrI	12961	13094
chrI	13624	13728
chrI	18243	18381
chrI	19394	19583
chrI	22136	22318


In [9]:
# Let's check the number of peaks from each file (after we removed the peaks from the 2 micron plasmid)
wc -l data/Gcn4_ChIP_Rawal_et_al.bed \
      data/Gcn4_motifs_Rawal_et_al.bed \
      data/Gcn4_SELEX_?.bed

     546 data/Gcn4_ChIP_Rawal_et_al.bed
    1748 data/Gcn4_motifs_Rawal_et_al.bed
     144 data/Gcn4_SELEX_1.bed
    2227 data/Gcn4_SELEX_2.bed
    2778 data/Gcn4_SELEX_3.bed
    7443 total


To do more complicated things, we'll use the `bedtools` package.

### Intro to bedtools
To install bedtools, first install Homebrew (https://brew.sh/):
```
/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
```
then install bedtools:
```
brew install bedtools
```

In [10]:
# Check that bedtools is correctly installed and added to your PATH
which bedtools

/usr/local/bin/bedtools


In [11]:
bedtools

bedtools is a powerful toolset for genome arithmetic.

Version:   v2.27.1
About:     developed in the quinlanlab.org and by many contributors worldwide.
Docs:      http://bedtools.readthedocs.io/
Code:      https://github.com/arq5x/bedtools2
Mail:      https://groups.google.com/forum/#!forum/bedtools-discuss

Usage:     bedtools <subcommand> [options]

The bedtools sub-commands include:

[ Genome arithmetic ]
    intersect     Find overlapping intervals in various ways.
    window        Find overlapping intervals within a window around an interval.
    closest       Find the closest, potentially non-overlapping interval.
    coverage      Compute the coverage over defined intervals.
    map           Apply a function to a column for each overlapping interval.
    genomecov     Compute the coverage over an entire genome.
    merge         Combine overlapping/nearby intervals into a single interval.
    cluster       Cluster (but don't merge) overlapping/nearby intervals.
    complement

### bedtools intersect

The `intersect` command is the workhorse of the `bedtools` suite. It compares two or more BED/BAM/VCF/GFF files and identifies all the regions in the gemome where the features in the two files overlap (that is, share at least one base pair in common).

![intersect](img/bedtools_intersect-01.png "bedtools intersect")

In [12]:
bedtools intersect


Tool:    bedtools intersect (aka intersectBed)
Version: v2.27.1
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A fea

: 1

The most useful options:
```
-wa    Write the original entry in A for each overlap.
-v     Only report those entries in A that have _no overlaps_ with B.
-wb    Write the original entry in B for each overlap.
```

In [13]:
# Check how many of the ChIP-seq peaks are found in a SELEX round
bedtools intersect -wa -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | wc -l

      83


In [14]:
# Check how many of the ChIP-seq peaks are NOT found in a SELEX round
bedtools intersect -v -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | wc -l

     465


In [15]:
# Total number of ChIP-seq peaks
cat data/Gcn4_ChIP_Rawal_et_al.bed | wc -l

     546


That is strange!  
83 + 465 is not equal to 546...

Why is this happening?  
This can happen if a peak from A overlapps with more peaks from B, in which case this peak is reported multiple times by bedtools intersect.  
Check the number of peaks from B that overlap with each peak from A.

In [16]:
bedtools intersect -wa -c -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | \
    sort -rn -t $'\t' -k4,4

chrXIII	608876	610041	3
chrXVI	747134	747587	1
chrXVI	625087	625563	1
chrXVI	497283	497717	1
chrXVI	24781	25501	1
chrXV	771175	771694	1
chrXV	731686	732110	1
chrXV	62085	62473	1
chrXV	58336	58828	1
chrXV	405694	406046	1
chrXV	397884	398436	1
chrXV	257104	257491	1
chrXV	218619	219290	1
chrXV	107470	107837	1
chrXIV	590753	591213	1
chrXIV	426654	427377	1
chrXIV	222019	222438	1
chrXIV	201443	201923	1
chrXIV	150442	150909	1
chrXIII	845351	845800	1
chrXIII	825991	826397	1
chrXIII	667719	668265	1
chrXIII	637087	637448	1
chrXIII	440776	441326	1
chrXII	91526	91975	1
chrXII	839314	839785	1
chrXII	499808	500260	1
chrXII	444579	445091	1
chrXII	254563	255033	1
chrXII	204781	205242	1
chrXII	1035473	1035900	1
chrXI	448446	448816	1
chrXI	408107	408519	1
chrXI	335687	336175	1
chrXI	273823	274312	1
chrXI	223932	224369	1
chrX	691533	691970	1
chrX	686373	686791	1
chrX	632973	633508	1
chrX	416221	416580	1
chrX	298499	298847	1
chrX	268216	268798	1
chrVIII	491862	492300	1
chrVIII	431582	431996	1
chrVIII	2992

chrVII	1077875	1078209	0
chrVII	1051957	1052200	0
chrVII	1025498	1025984	0
chrVI	95490	95824	0
chrVI	80174	80551	0
chrVI	267136	267611	0
chrVI	263385	263797	0
chrVI	250397	250842	0
chrVI	210127	210763	0
chrVI	200816	201045	0
chrVI	131045	131293	0
chrVI	118341	118637	0
chrV	77619	77853	0
chrV	68916	69335	0
chrV	540282	540757	0
chrV	485875	486106	0
chrV	465557	465795	0
chrV	435636	435973	0
chrV	434561	434958	0
chrV	419240	419472	0
chrV	337635	338010	0
chrV	33541	34065	0
chrV	303605	304068	0
chrV	286709	287305	0
chrV	271245	271619	0
chrV	265719	266256	0
chrV	257997	258492	0
chrV	244997	245226	0
chrV	229539	229860	0
chrV	226458	226869	0
chrV	201889	202219	0
chrV	16022	16295	0
chrV	152746	153178	0
chrV	146167	146423	0
chrV	141039	141415	0
chrIX	86004	86227	0
chrIX	56341	56759	0
chrIX	420655	421138	0
chrIX	386577	386938	0
chrIX	384055	384379	0
chrIX	366820	367049	0
chrIX	34482	35013	0
chrIX	300095	300442	0
chrIX	258260	258495	0
chrIX	248769	249449	0
chrIX	237921	238297	0
chrIX	212144	212560	

We find that a peak from Gcn4_ChIP_Rawal_et_al.bed (chrXIII:608876-610041) overlaps with 3 peaks from Gcn4_SELEX_1.bed. That peak will be reported by `bedtools` 3 times in the resulting intersection list.

In [17]:
# Check the above statement
bedtools intersect -wa -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | \
    grep "chrXIII\t608876\t610041"

chrXIII	608876	610041
chrXIII	608876	610041
chrXIII	608876	610041


To correct this default behavior of bedtools, use the `-u` argument  
```
-u    Write the original A entry _once_ if _any_ overlaps found in B.
```

In [18]:
# Check the effect of -u argument
bedtools intersect -wa -u -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | \
    grep "chrXIII\t608876\t610041"

chrXIII	608876	610041


Repeat the intersections, using the -u argument that reports unique peaks after intersection.

In [19]:
printf "Total number of ChIP-seq peaks:"
cat data/Gcn4_ChIP_Rawal_et_al.bed | wc -l

printf "Number of ChIP-seq peaks detected in SELEX:"
bedtools intersect -wa -u -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | wc -l

printf "Number of ChIP-seq peaks NOT detected in SELEX:"
bedtools intersect -v -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_1.bed | wc -l

Total number of ChIP-seq peaks:     546
Number of ChIP-seq peaks detected in SELEX:      81
Number of ChIP-seq peaks NOT detected in SELEX:     465


##### Exercise
Repeat these computations with the SELEX peaks detected in round 2 & 3
```
bedtools intersect -wa -u -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_2.bed | wc -l
bedtools intersect -v -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_2.bed | wc -l

bedtools intersect -wa -u -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_3.bed | wc -l
bedtools intersect -v -a data/Gcn4_ChIP_Rawal_et_al.bed -b data/Gcn4_SELEX_3.bed | wc -l
```

You should obtain the following results:  
For SELEX round 2, 446 ChIP-seq peaks were detected, 100 peaks were not detected.  
For SELEX round 3, 475 ChIP-seq peaks were detected, 71 peaks were not detected.

Next, let's look closer at the peaks detected in SELEX round 3. Let's split them into a list of peaks that were detected in ChIP-seq experiments, and a list of peaks that were NOT detected in ChIP-seq experiments.

In [20]:
printf "Total number of SELEX peaks:"
cat data/Gcn4_SELEX_3.bed | wc -l

printf "Number of SELEX peaks detected in ChIP-seq:"
bedtools intersect -wa -u -a data/Gcn4_SELEX_3.bed -b data/Gcn4_ChIP_Rawal_et_al.bed \
    > data/SELEX_3_peaks_found_in_ChIP.bed
cat data/SELEX_3_peaks_found_in_ChIP.bed | wc -l

printf "Number of SELEX peaks detected in ChIP-seq:"
bedtools intersect -v -a data/Gcn4_SELEX_3.bed -b data/Gcn4_ChIP_Rawal_et_al.bed \
    > data/SELEX_3_peaks_NOT_found_in_ChIP.bed
cat data/SELEX_3_peaks_NOT_found_in_ChIP.bed | wc -l

Total number of SELEX peaks:    2778
Number of SELEX peaks detected in ChIP-seq:     499
Number of SELEX peaks detected in ChIP-seq:    2279


In [21]:
ls -lh data/SELEX_3_peaks_*.bed

-rw-r--r--  1 cherejirv  NIH\Domain Users    45K Mar 26 16:51 data/SELEX_3_peaks_NOT_found_in_ChIP.bed
-rw-r--r--  1 cherejirv  NIH\Domain Users   9.9K Mar 26 16:51 data/SELEX_3_peaks_found_in_ChIP.bed


### Motif search using MEME

First we need to get the DNA sequence corresponding to the above BED files.  
We can download the sacCer3 fasta file from here:  
http://hgdownload.cse.ucsc.edu/goldenPath/sacCer3/bigZips/chromFa.tar.gz

In [22]:
curl http://hgdownload.cse.ucsc.edu/goldenPath/sacCer3/bigZips/chromFa.tar.gz \
    -o data/chromFa.tar.gz
    
# List the files from the archive    
tar -tf data/chromFa.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3730k  100 3730k    0     0  2156k      0  0:00:01  0:00:01 --:--:-- 2155k
chrI.fa
chrII.fa
chrIII.fa
chrIV.fa
chrIX.fa
chrM.fa
chrV.fa
chrVI.fa
chrVII.fa
chrVIII.fa
chrX.fa
chrXI.fa
chrXII.fa
chrXIII.fa
chrXIV.fa
chrXV.fa
chrXVI.fa


In [23]:
# Extract FASTA files
mkdir -p temp
tar -C temp -zxf data/chromFa.tar.gz

# Create a single FASTA file that contains the DNA sequences of all chromosomes
cat temp/* > data/sacCer3.fa

# Remove the temp folder and the tar.gz file
rm -r temp
rm data/chromFa.tar.gz

### bedtools gatfasta
Next we will use `bedtools getfasta` to obtain the DNA sequences corresponding to the found peaks.  
![getfasta](img/getfasta-glyph.png "bedtools getfasta")

But first, let's resize the intervals from the BED files to a constant width of 101 bp.

In [24]:
awk -F '\t' '{X=50; mid=(int($2)+int($3))/2; printf("%s\t%d\t%d\n", $1, (mid-X<0?0:mid-X),mid+X);}' \
    data/SELEX_3_peaks_found_in_ChIP.bed \
    > data/resized_SELEX_3_peaks_found_in_ChIP.bed
    
awk -F '\t' '{X=50; mid=(int($2)+int($3))/2; printf("%s\t%d\t%d\n", $1, (mid-X<0?0:mid-X),mid+X);}' \
    data/SELEX_3_peaks_NOT_found_in_ChIP.bed \
    > data/resized_SELEX_3_peaks_NOT_found_in_ChIP.bed

Now, use `bedtools getfasta` to obtain the corresponding DNA sequences (necessary for MEME)

In [25]:
bedtools getfasta -fi data/sacCer3.fa -bed data/resized_SELEX_3_peaks_found_in_ChIP.bed \
    > data/resized_SELEX_3_peaks_found_in_ChIP.fa
bedtools getfasta -fi data/sacCer3.fa -bed data/resized_SELEX_3_peaks_NOT_found_in_ChIP.bed \
    > data/resized_SELEX_3_peaks_NOT_found_in_ChIP.fa

index file data/sacCer3.fa.fai not found, generating...


We generated the fasta files that contain the DNA sequences corresponding to the two classes of Gcn4 peaks: peaks that were found in ChIP-seq data, and peaks that were only found in SELEX-seq data but not in ChIP-seq.  

Now, we can use `MEME` to generate the motifs corresponding to these two classes of peaks:  
Step 1: Open a browser and go to http://meme-suite.org/tools/meme  
The website should look like this:
![MEME](img/MEME.png "MEME tool")

Step 2: In the section **Input the primary sequences** click on **Choose File** and upload your fasta file witht the DNA sequences (resized_SELEX_3_peaks_found_in_ChIP.fa or resized_SELEX_3_peaks_NOT_found_in_ChIP.fa)  
Step 3: Enter your email address, and a useful job description (this is how you'll be able to track the job results later)  
Step 4: Click on **Start Search**

When `MEME` has finished its thing, you'll receive an email with a link to the results. Among other things you'll find there the sequence motif with the corresponding position weight matrix.

### Further reading resources:  
`bedtools` manual: https://bedtools.readthedocs.io/en/latest/index.html  
`bedtools` tutorial: http://quinlanlab.org/tutorials/bedtools/bedtools.html  
`MACS2` tutorial: https://github.com/taoliu/MACS/wiki/Call-differential-binding-events  
ChIP-seq tutorial: http://ginolhac.github.io/chip-seq/peak/  